# Adding variables to model

In [1]:
import pandas as pd
import ETL as etl
import numpy as np

# Parameters
SECOPI_PROCESS_API = 'f789-7hwg'
SECOPII_PROCESS_API = 'jbjy-vk9h'
SECOPI_ADDITIONS_API = '7fix-nd37'
SECOPI_PUNISHMENT_API = '4n4q-k399'

KEY = '29f7zgixdbbfu4gi5m251frq1'

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## SECOP I

In [2]:
procesos_data = pd.read_csv('data/collected_obra_data.csv',
                 dtype = {'ID_ADJUDICACION': str},
                 parse_dates=['START_DATE','CONTRACT_DATE'])

procesos_secop_i_data = procesos_data[-procesos_data['GROUP'].str.contains('SECOP II')].copy()

procesos_secop_i_data['HAVE_DEVIATION'] = (procesos_secop_i_data['COST_DEVIATION']  > 0) | (procesos_secop_i_data['TIME_DEVIATION'] > 0)

### First addition time

In [4]:
ids = list(set(procesos_secop_i_data['ID_ADJUDICACION']))

def parse_to_list(ls):
    ids = [ str(i) for i in ls]
    ids = "'" + "','".join(ids) + "'"
    return ids

ids = parse_to_list(ids)

with open("bin/queries/request_first_adding_date.sql", "r", encoding="utf8") as query_file:
    query = query_file.read()

query = query.format(LIST_UID = ids)

temp_additions_data = etl.extract_data(query, id_data=SECOPI_ADDITIONS_API, api_key=None)

temp_additions_data.head()

El numero de contratos extraidos: 666


,ID_ADJUDICACION,NUM_ADDITION,NUM_ADDITION_VALUE,NUM_ADDITION_TIME,FIRST_ADDITION_AT,FIRST_ADDITION_TIME_AT,FIRST_ADDITION_VALUE_AT
0,10062354,2,0,2,2020-06-25T00:00:00.000,2020-06-25T00:00:00.000,NaN
1,10062360,3,0,3,2020-09-25T00:00:00.000,2020-09-25T00:00:00.000,NaN
2,10159407,4,0,4,2020-12-30T00:00:00.000,2020-12-30T00:00:00.000,NaN
3,10285913,1,0,1,2021-08-25T00:00:00.000,2021-08-25T00:00:00.000,NaN
4,10289495,1,1,0,2020-12-10T00:00:00.000,NaN,2020-12-10T00:00:00.000


In [5]:
temp_additions_data['FIRST_ADDITION_AT'] = pd.to_datetime(temp_additions_data['FIRST_ADDITION_AT'])
procesos_secop_i_data = pd.merge(procesos_secop_i_data, temp_additions_data, 
                                    how='left', on='ID_ADJUDICACION')

procesos_secop_i_data['DAY_DIFF_FIRST_ADDITION'] = (procesos_secop_i_data['FIRST_ADDITION_AT'] - procesos_secop_i_data['START_DATE']) / np.timedelta64(1, 'D')
procesos_secop_i_data['RATIO_FIRST_ADDITION'] = procesos_secop_i_data['DAY_DIFF_FIRST_ADDITION'] / procesos_secop_i_data['ORIGINAL_DEADLINE']

bins = [0, 0.25, 0.5, 0.75, 1, float('inf')]  # Define your bin edges
labels = ['very early', 'early', 'medium', 'late', 'very late']  # Assign labels to bins

# Create a new categorical column based on the bins
procesos_secop_i_data['FIRST_ADITION_TIME_GROUP'] = pd.cut(procesos_secop_i_data['RATIO_FIRST_ADDITION'], bins=bins, labels=labels, right=False)

procesos_secop_i_data['FIRST_ADITION_TIME_GROUP'] = procesos_secop_i_data['FIRST_ADITION_TIME_GROUP'].cat.add_categories('NA').fillna('NA')

procesos_secop_i_data[['URLPROCESO','START_DATE', 'ORIGINAL_DEADLINE', 'FIRST_ADDITION_AT', 'DAY_DIFF_FIRST_ADDITION', 'FIRST_ADITION_TIME_GROUP']].head(10)

,URLPROCESO,START_DATE,ORIGINAL_DEADLINE,FIRST_ADDITION_AT,DAY_DIFF_FIRST_ADDITION,FIRST_ADITION_TIME_GROUP
0,{'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=19-1-205607'},2019-12-04,60.0,2020-11-27,359.0,very late
1,{'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=16-1-163706'},2016-12-14,360.0,2018-07-19,582.0,very late
2,{'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=19-1-200502'},2019-07-14,120.0,NaT,NaN,NA
3,{'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=18-1-195988'},2019-01-29,240.0,NaT,NaN,NA
4,{'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=15-1-143528'},2015-09-08,90.0,NaT,NaN,NA
5,{'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=15-1-139460'},2015-12-15,270.0,2016-09-12,272.0,very late
6,{'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=19-21-13283'},2019-10-17,73.0,NaT,NaN,NA
7,{'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=18-21-7179'},2019-02-05,150.0,NaT,NaN,NA
8,{'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=16-1-161975'},2016-11-03,540.0,2018-04-12,525.0,late
9,{'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=15-1-150888'},2016-02-12,930.0,2018-09-12,943.0,very late


: 

In [169]:
def summary(x):
    data = {}

    data['unique contracts'] = x['CONTRACT_ID'].size
    data['contracts with cost deviation'] = x[x['COST_DEVIATION'] > 0]['CONTRACT_ID'].size
    data['contracts with time deviation'] = x[x['TIME_DEVIATION'] > 0]['CONTRACT_ID'].size
    data['contracts with both deviation'] = x[(x['TIME_DEVIATION'] > 0) | (x['COST_DEVIATION'] > 0)]['CONTRACT_ID'].size
    data['value total'] = x['CONTRACT_VALUE'].sum()
    data['value average'] = x['CONTRACT_VALUE'].mean()
    data['cost deviation total'] = x['COST_DEVIATION'].sum()
    data['cost deviation average'] = x['COST_DEVIATION'].mean()
    data['ratio deviation / cost'] = data['cost deviation total'] / data['value total']

    return pd.Series(data)

procesos_secop_i_data.groupby('FIRST_ADITION_TIME_GROUP').apply(summary)

C:\Users\nicolas.arrieta\AppData\Local\Temp\ipykernel_10008\1876086017.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  procesos_secop_i_data.groupby('FIRST_ADITION_TIME_GROUP').apply(summary)


,unique contracts,contracts with cost deviation,contracts with time deviation,contracts with both deviation,value total,value average,cost deviation total,cost deviation average,ratio deviation / cost
FIRST_ADITION_TIME_GROUP,,,,,,,,,
very early,16.0,11.0,14.0,16.0,5.459371e+04,3412.106903,2.323900,0.145244,4.256718e-05
early,24.0,21.0,14.0,24.0,7.872151e+04,3280.062899,6.131950,0.255498,7.789421e-05
medium,42.0,37.0,27.0,42.0,2.013048e+05,4792.971580,11.569468,0.275464,5.747239e-05
late,194.0,136.0,168.0,193.0,1.645989e+06,8484.480391,43.396276,0.223692,2.636486e-05
very late,385.0,212.0,359.0,384.0,2.454792e+06,6376.084257,52.032987,0.135151,2.119649e-05
NA,708.0,2.0,4.0,5.0,2.843250e+06,4015.889728,0.353741,0.000500,1.244144e-07


In [170]:
procesos_secop_i_data.groupby('NUM_ADDITION').apply(summary)

,unique contracts,contracts with cost deviation,contracts with time deviation,contracts with both deviation,value total,value average,cost deviation total,cost deviation average,ratio deviation / cost
NUM_ADDITION,,,,,,,,,
1,330.0,181.0,257.0,329.0,972759.595134,2947.756349,47.702706,0.144554,4.903854e-05
12,1.0,1.0,1.0,1.0,29387.520087,29387.520087,0.531379,0.531379,1.808179e-05
13,1.0,1.0,1.0,1.0,128987.553055,128987.553055,0.216858,0.216858,1.681229e-06
2,180.0,126.0,175.0,180.0,981937.639384,5455.209108,36.422633,0.202348,3.709261e-05
3,76.0,47.0,75.0,75.0,486706.700879,6404.035538,12.183538,0.160310,2.503261e-05
4,40.0,33.0,39.0,40.0,498917.522875,12472.938072,11.940986,0.298525,2.393379e-05
5,18.0,14.0,18.0,18.0,290524.184136,16140.232452,3.832493,0.212916,1.319165e-05
6,10.0,7.0,10.0,10.0,271194.504956,27119.450496,1.260669,0.126067,4.648578e-06
7,5.0,4.0,5.0,5.0,269762.761372,53952.552274,0.563253,0.112651,2.087956e-06


### PUNISHMENT

In [171]:
ids_entidad = list(set(procesos_secop_i_data['NIT_ENTIDAD']))
ids_contratistas = list(set(procesos_secop_i_data['ID_CONTRATISTA']))

ids_entidad = parse_to_list(ids_entidad)
ids_contratistas = parse_to_list(ids_contratistas)

with open("bin/queries/request_punishment.sql", "r", encoding="utf8") as query_file:
    query = query_file.read()

query = query.format(NIT_ENTIDAD_LIST = ids_entidad, NIT_CONTRATISTAS_LIST = ids_contratistas)

temp_punishment_data = etl.extract_data(query, id_data=SECOPI_PUNISHMENT_API, api_key=None)

temp_punishment_data['PENALTY_DATE'] = pd.to_datetime(temp_punishment_data['PENALTY_DATE'])

temp_punishment_data.head()

El numero de contratos extraidos: 706


,PENALTY_ID,NIT_ENTIDAD,ID_CONTRATISTA,PENALTY_VALUE,PENALTY_DATE
0,147-2020,800100532-8,900.659.669-0,1658082,2021-05-06
1,CM 001 DE 2019,800096580-4,901.266.627-8,22400000,2020-03-23
2,SIN NUMERO,800100134-1,9001209218,74000000,2016-07-13
3,06-334-2015,890204646-3,900260321,999055,2017-12-18
4,06-045-2019,890204646-3,890935513,15903790,2019-06-26


In [172]:
# entidad
temp_punishment_entidad_data = pd.merge(procesos_secop_i_data[['CONTRACT_ID','NIT_ENTIDAD','CONTRACT_DATE','END_DATE']], temp_punishment_data, 
                                    how='inner', on='NIT_ENTIDAD')

def penalty_summary(x,pref):
    data = {}

    data['NUM_PENALTIES' + pref] = x['PENALTY_ID'].size
    #data['VAL_PENALTIES'] = x['PENALTY_VALUE'].sum()
    return pd.Series(data)

temp_punishment_entidad_active = temp_punishment_entidad_data[(temp_punishment_entidad_data['END_DATE'] > temp_punishment_entidad_data['PENALTY_DATE']) &
                                                            (temp_punishment_entidad_data['PENALTY_DATE'] > temp_punishment_entidad_data['CONTRACT_DATE'])]\
                                                                .drop_duplicates().groupby('CONTRACT_ID').apply(penalty_summary, '_ENTIDAD_ACTIVE')

temp_punishment_entidad_last = temp_punishment_entidad_data[(temp_punishment_entidad_data['CONTRACT_DATE'] > temp_punishment_entidad_data['PENALTY_DATE']) &
                                                            (temp_punishment_entidad_data['PENALTY_DATE'] > (temp_punishment_entidad_data['CONTRACT_DATE'] - pd.DateOffset(years=1)))]\
                                                                .drop_duplicates().groupby('CONTRACT_ID').apply(penalty_summary, '_ENTIDAD_LAST_Y')

In [173]:
procesos_secop_i_data.reset_index(drop = True, inplace = True)
temp_punishment_entidad_active.reset_index(inplace = True)
temp_punishment_entidad_last.reset_index(inplace = True)
#union new rows
procesos_secop_i_data = pd.merge(procesos_secop_i_data, 
                                 temp_punishment_entidad_active, 
                                 how='left', on='CONTRACT_ID')
procesos_secop_i_data = pd.merge(procesos_secop_i_data, 
                                 temp_punishment_entidad_last, 
                                 how='left', on='CONTRACT_ID')

In [174]:
# contratistas
temp_punishment_contratista_data = pd.merge(procesos_secop_i_data[['CONTRACT_ID','ID_CONTRATISTA','CONTRACT_DATE','END_DATE']], temp_punishment_data, 
                                    how='inner', on='ID_CONTRATISTA')

temp_punishment_contratista_active = temp_punishment_contratista_data[(temp_punishment_contratista_data['END_DATE'] > temp_punishment_contratista_data['PENALTY_DATE']) &
                                                            (temp_punishment_contratista_data['PENALTY_DATE'] > temp_punishment_contratista_data['CONTRACT_DATE'])]\
                                                                .drop_duplicates().groupby('CONTRACT_ID').apply(penalty_summary, '_CONTRATISTA_ACTIVE')

temp_punishment_contratista_last = temp_punishment_contratista_data[(temp_punishment_contratista_data['CONTRACT_DATE'] > temp_punishment_contratista_data['PENALTY_DATE']) &
                                                            (temp_punishment_contratista_data['PENALTY_DATE'] > (temp_punishment_contratista_data['CONTRACT_DATE'] - pd.DateOffset(years=1)))]\
                                                                .drop_duplicates().groupby('CONTRACT_ID').apply(penalty_summary, '_CONTRATISTA_LAST_Y')

In [175]:
#union new rows
temp_punishment_contratista_last.reset_index(inplace = True)

procesos_secop_i_data = pd.merge(procesos_secop_i_data, 
                                 temp_punishment_contratista_last, 
                                 how='left', on='CONTRACT_ID')

In [181]:
new_columns = ['NUM_PENALTIES_ENTIDAD_ACTIVE','NUM_PENALTIES_ENTIDAD_LAST_Y', 'NUM_PENALTIES_CONTRATISTA_LAST_Y']
procesos_secop_i_data[new_columns] = procesos_secop_i_data[new_columns].fillna(0)

In [183]:
procesos_secop_i_data[['CONTRACT_ID','START_DATE','CONTRACT_VALUE','ADDITIONAL_COST',
                       'NUM_PENALTIES_ENTIDAD_ACTIVE','NUM_PENALTIES_ENTIDAD_LAST_Y', 
                       'NUM_PENALTIES_CONTRATISTA_LAST_Y']].head()

,CONTRACT_ID,START_DATE,CONTRACT_VALUE,ADDITIONAL_COST,NUM_PENALTIES_ENTIDAD_ACTIVE,NUM_PENALTIES_ENTIDAD_LAST_Y,NUM_PENALTIES_CONTRATISTA_LAST_Y
0,19-1-205607-10520942,2019-12-04,1309.283977,301.963131,0.0,0.0,0.0
1,16-1-163706-5387695,2016-12-14,33866.017903,6740.014822,1.0,0.0,0.0
2,19-1-200502-9032665,2019-07-14,771.730201,0.000000,0.0,0.0,0.0
3,18-1-195988-8133851,2019-01-29,1238.482714,0.000000,0.0,0.0,0.0
4,15-1-143528-3891257,2015-09-08,519.902870,0.000000,0.0,0.0,0.0


In [ ]:
procesos_secop_i_data.info()

In [189]:
def summary2(x):
    data = {}
    total_cost = x['CONTRACT_VALUE'].sum()
    total_projects = x['CONTRACT_ID'].size
    data['num contratos'] = total_projects = x['CONTRACT_ID'].size
    data['avg project cost'] = total_cost / total_projects
    data['avg penalties to entity during project'] = x['NUM_PENALTIES_ENTIDAD_ACTIVE'].mean()
    data['var penalties to entity during project'] = x['NUM_PENALTIES_ENTIDAD_ACTIVE'].std()
    data['avg penalties to entity last year'] = x['NUM_PENALTIES_ENTIDAD_LAST_Y'].mean()
    data['var penalties to entity last year'] = x['NUM_PENALTIES_ENTIDAD_LAST_Y'].std()

    return pd.Series(data)

procesos_secop_i_data.groupby('HAVE_DEVIATION').apply(summary2)

,num contratos,avg project cost,avg penalties to entity during project,var penalties to entity during project,avg penalties to entity last year,var penalties to entity last year
HAVE_DEVIATION,,,,,,
False,705.0,4025.840066,3.303546,10.859579,6.374468,18.151945
True,664.0,6687.401117,5.390060,16.430941,6.721386,18.703830
